Terminal -> New Terminal

python3 -m venv venv

source venv/bin/activate

Select Kernel -> venv

pip install evtx

pip freeze > requirements.txt

pip install -r requirements.txt (quando for instalar)

In [1]:
import os

In [2]:
#cwd (current working directory)
caminho_para_diretorio_atual = os.getcwd()
caminho_para_diretorio_atual

'/home/mx/0 - shared/1 - Pesquisa/UFU_IC/workspace/jupyter'

In [3]:
caminho_para_arquivos_evtx = "/home/mx/0 - shared/1 - Pesquisa/UFU_IC/workspace/Logs/"
#chdir (change directory)
os.chdir(caminho_para_arquivos_evtx)
os.getcwd()

'/home/mx/0 - shared/1 - Pesquisa/UFU_IC/workspace/Logs'

In [4]:
import glob
for nome_do_arquivo_evtx in glob.glob('*.evtx'):
    print(nome_do_arquivo_evtx)

Application.evtx
System.evtx


In [5]:
from evtx import PyEvtxParser

for nome_do_arquivo_evtx in glob.glob('*.evtx'):
    conjunto_de_registros_do_evtx = PyEvtxParser(caminho_para_arquivos_evtx+nome_do_arquivo_evtx)
    for registro in conjunto_de_registros_do_evtx.records():
        print(registro)
        break

{'event_record_id': 140946, 'timestamp': '2024-03-22 23:19:57.959605 UTC', 'data': '<?xml version="1.0" encoding="utf-8"?>\n<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event">\n  <System>\n    <Provider Name="Microsoft-Windows-Security-SPP" Guid="{E23B33B0-C8C9-472C-A5F9-F2BDFEA0F156}" EventSourceName="Software Protection Platform Service">\n    </Provider>\n    <EventID Qualifiers="16384">1003</EventID>\n    <Version>0</Version>\n    <Level>4</Level>\n    <Task>0</Task>\n    <Opcode>0</Opcode>\n    <Keywords>0x80000000000000</Keywords>\n    <TimeCreated SystemTime="2024-03-22T23:19:57.959605Z">\n    </TimeCreated>\n    <EventRecordID>140946</EventRecordID>\n    <Correlation>\n    </Correlation>\n    <Execution ProcessID="0" ThreadID="0">\n    </Execution>\n    <Channel>Application</Channel>\n    <Computer>Athena</Computer>\n    <Security>\n    </Security>\n  </System>\n  <EventData>\n    <Data>0ff1ce15-a989-479d-af46-f275c6370663,\n1: d450596f-894d-49e0-966a-fd39ed4c

In [6]:
registro['data']

'<?xml version="1.0" encoding="utf-8"?>\n<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event">\n  <System>\n    <Provider Name="Microsoft-Windows-DistributedCOM" Guid="{1B562E86-B7AA-4131-BADC-B6F3A001407E}" EventSourceName="DCOM">\n    </Provider>\n    <EventID Qualifiers="0">10016</EventID>\n    <Version>0</Version>\n    <Level>3</Level>\n    <Task>0</Task>\n    <Opcode>0</Opcode>\n    <Keywords>0x8080000000000000</Keywords>\n    <TimeCreated SystemTime="2024-04-04T21:30:01.002528Z">\n    </TimeCreated>\n    <EventRecordID>155520</EventRecordID>\n    <Correlation ActivityID="6095A4FD-6545-4F21-8FFA-CD64750B351D">\n    </Correlation>\n    <Execution ProcessID="1104" ThreadID="16976">\n    </Execution>\n    <Channel>System</Channel>\n    <Computer>Athena</Computer>\n    <Security UserID="S-1-5-21-3179950460-3881450488-1956027464-1001">\n    </Security>\n  </System>\n  <EventData>\n    <Data Name="param1">application-specific</Data>\n    <Data Name="param2">Local</Data>\

O código busca uma string que começa com <Level>, seguida de qualquer conteúdo (capturado por (.*?)), e termina com </Level> dentro da string registro['data'].

In [7]:
import re
re.search(r'<Level>(.*?)</Level>', registro['data'])

<re.Match object; span=(333, 349), match='<Level>3</Level>'>

Esta parte do código acessa o primeiro grupo de captura ((.*?)) do objeto de correspondência retornado por re.search(). Em expressões regulares, os grupos de captura são definidos colocando partes do padrão entre parênteses. Como (.*?) é o primeiro (e único) grupo de captura no padrão, .group(1) recupera o conteúdo correspondido por este grupo.

In [8]:
re.search(r'<Level>(.*?)</Level>', registro['data']).group(1)

'3'

In [9]:
event_level = re.search(r'<Level>(.*?)</Level>', registro['data']).group(1)
event_level

'3'

# Event_level 1 e 2 são falhas

In [10]:
# if((("Application" in nome_do_arquivo_evtx) & (event_level == "2")) | (("System" in nome_do_arquivo_evtx) & ((event_level == "2") | (event_level == "1")))):

Como refatorar o código

In [11]:
# conditions = [
#     ("Application" in nome_do_arquivo_evtx and event_level == "2"),
#     ("System" in nome_do_arquivo_evtx and event_level in ("1", "2"))
# ]

# if any(conditions):
#     # Your code here


### Eventos que não saõ de falhas, mas possuem Event_level 1 e 2

In [12]:
import pandas as pd

os.chdir(caminho_para_diretorio_atual)
df_non_failure_events = pd.read_csv("1_non_failure_events.csv", encoding='utf-8-sig', engine='python')
df_non_failure_events

,SourceName,EventID
0,trend micro officescan,900
1,eventlog,6008
2,tcpip,4199
3,server,2505
4,netbt,4321
5,netbt,4319
6,microsoft-windows-kernel-power,41


In [13]:
from collections import defaultdict


dict_non_failure_events = defaultdict(list)
for k, v in zip(df_non_failure_events.SourceName, df_non_failure_events.EventID):
    dict_non_failure_events[k].append(v)
dict_non_failure_events

defaultdict(list,
            {'trend micro officescan': [900],
             'eventlog': [6008],
             'tcpip': [4199],
             'server': [2505],
             'netbt': [4321, 4319],
             'microsoft-windows-kernel-power': [41]})

# Código Completo

In [14]:
class Event(object):
    def __init__(
        self,
        Filename,
        ID,
        SourceName,
        Provider_Guid,
        EventID,
        EventID_Qualifiers,
        Version,
        Level,
        Task,
        Opcode,
        Keywords,
        TimeCreated,
        EventRecordID,
        Correlation_ActivityID,
        Correlation_RelatedActivityID,
        ProcessID,
        ThreadID,
        Channel,
        Computer,
        UserID,
        Data_Event_Name,
        Data_Fields,
        Data,
        UserData_Flag
    ):
        self.Filename = Filename
        self.ID = ID
        self.SourceName = SourceName
        self.Provider_Guid = Provider_Guid
        self.EventID = EventID
        self.EventID_Qualifiers = EventID_Qualifiers
        self.Version = Version
        self.Level = Level
        self.Task = Task
        self.Opcode = Opcode
        self.Keywords = Keywords
        self.TimeCreated = TimeCreated
        self.EventRecordID = EventRecordID
        self.Correlation_ActivityID = Correlation_ActivityID
        self.Correlation_RelatedActivityID = Correlation_RelatedActivityID
        self.ProcessID = ProcessID
        self.ThreadID = ThreadID
        self.Channel = Channel
        self.Computer = Computer
        self.UserID = UserID
        self.Data_Event_Name = Data_Event_Name
        self.Data_Fields = Data_Fields
        self.Data = Data
        self.UserData_Flag = UserData_Flag

In [15]:
def parseXMLtoString(root, string_search):
    result = root.xpath(string_search)
    if  ("Data" not in string_search) and (result):
        result = str(result[0])

    return result

In [17]:
from lxml import etree
import xmltodict

os.chdir(caminho_para_arquivos_evtx)
listaDe_eventos_dos_arquivos_evtx = []
i = 0
UserData_Flag = 0
for nome_do_arquivo_evtx in glob.glob('*.evtx'):
    conjunto_de_registros_do_evtx = PyEvtxParser(caminho_para_arquivos_evtx+nome_do_arquivo_evtx)
    for registro in conjunto_de_registros_do_evtx.records():
        event_level = re.search(r'<Level>(.*?)</Level>', registro['data']).group(1)
        event_id = re.search(r'<EventID(?:[^>]*)>(\d+)</EventID>', registro['data']).group(1)
        #Event_level 1 e 2 são falhas
        if((("Application" in nome_do_arquivo_evtx) & (event_level == "2")) | (("System" in nome_do_arquivo_evtx) & ((event_level == "2") | (event_level == "1")))):
            i = i + 1
            Filename = nome_do_arquivo_evtx.split("_")[0]

            #Em produção
            # Filename = nome_do_arquivo_evtx


            ID = i

            #Removendo "<?xml version="1.0" encoding="utf-8"?>" do evento
            line_filtered_1 = re.sub(r'<\?xml([\s\S]*?)>\n','',registro['data'])
            #Removendo "xmlns="http://schemas.microsoft.com/win/2004/08/events/event"" do evento
            line_filtered_2 = re.sub(r' xmlns=\"([\s\S]*?)\"','',line_filtered_1)

            try:
                evento_estrutura_xml = etree.XML(line_filtered_2)

                evento_estrutura_dict = xmltodict.parse(line_filtered_2)


            except Exception as e:
                with open('D:\\Dropbox\\Jupyter\\2023_Engine\\Output\\exception.txt', 'a', encoding='utf-8') as f:     
                    f.write(nome_do_arquivo_evtx+"_"+str(ID)+"_"+str(e))
                    f.write("\n")
                    continue
            else:

                evento_estrutura_dict__tag_System = evento_estrutura_dict.get("Event").get("System")

                SourceName = evento_estrutura_dict__tag_System.get("Provider").get("@Name")
                if(SourceName is None):
                    SourceName = evento_estrutura_dict__tag_System.get("Provider").get("@EventSourceName")


                Provider_Guid = evento_estrutura_dict__tag_System.get("Provider").get("@Guid")

                EventID = evento_estrutura_dict__tag_System.get("EventID")
                EventID_Qualifiers = None
                if (type(EventID) != str):
                    EventID = evento_estrutura_dict__tag_System.get("EventID").get("#text")
                    EventID_Qualifiers = evento_estrutura_dict__tag_System.get("EventID").get("@Qualifiers")

                # Removendo elementos que não são de falha.
                if(SourceName.lower() in dict_non_failure_events.keys()):         
                    listaDe_EventIDs = dict_non_failure_events[SourceName.lower()]         
                    if(int(EventID) in listaDe_EventIDs):             
                        continue

                Version = evento_estrutura_dict__tag_System.get("Version")
                Level = evento_estrutura_dict__tag_System.get("Level")
                Task = evento_estrutura_dict__tag_System.get("Task")
                Opcode = evento_estrutura_dict__tag_System.get("Opcode")
                Keywords = evento_estrutura_dict__tag_System.get("Keywords")
                TimeCreated = evento_estrutura_dict__tag_System.get("TimeCreated").get("@SystemTime")
                EventRecordID = evento_estrutura_dict__tag_System.get("EventRecordID")

                correlation = evento_estrutura_dict__tag_System.get("Correlation")
                Correlation_ActivityID = None
                Correlation_RelatedActivityID = None
                if(correlation is not None):
                    Correlation_ActivityID = evento_estrutura_dict__tag_System.get("Correlation").get("@ActivityID")
                    Correlation_RelatedActivityID = evento_estrutura_dict__tag_System.get("Correlation").get("@RelatedActivityID")

                execution = evento_estrutura_dict__tag_System.get("Execution")
                ProcessID = None
                ThreadID = None
                if(execution is not None):
                    ProcessID = evento_estrutura_dict__tag_System.get("Execution").get("@ProcessID")
                    ThreadID = evento_estrutura_dict__tag_System.get("Execution").get("@ThreadID")

                Channel = evento_estrutura_dict__tag_System.get("Channel")
                Computer = evento_estrutura_dict__tag_System.get("Computer")

                security = evento_estrutura_dict__tag_System.get("Security")
                UserID = None
                if(security is not None):
                    UserID = evento_estrutura_dict__tag_System.get("Security").get("@UserID")

                dataTag = evento_estrutura_xml.getchildren()[1].tag

                if (dataTag == 'ProcessingErrorData'):
                    Data_Fields = list(evento_estrutura_dict.get("Event").get("ProcessingErrorData").keys())
                    Data = list(evento_estrutura_dict.get("Event").get("ProcessingErrorData").values())
                    UserData_Flag = 0
                    Data_Event_Name = None

                elif (dataTag != 'EventData'):        
                    Data_Fields = list(evento_estrutura_dict.get("Event").get("UserData").get(list(evento_estrutura_dict.get("Event").get(dataTag).keys())[0]).keys())
                    Data = list(evento_estrutura_dict.get("Event").get("UserData").get(list(evento_estrutura_dict.get("Event").get(dataTag).keys())[0]).values())
                    UserData_Flag = 1
                    Data_Event_Name = None

                else:
                    UserData_Flag = 0
                    Data_Event_Name = None
                    evento_estrutura_dictEventData = evento_estrutura_dict.get("Event").get("EventData")
                    if(evento_estrutura_dictEventData is None):
                        Data_Fields = None
                        Data = None
                    else:
                        Data_Fields = (parseXMLtoString(evento_estrutura_xml, ".//Data/@Name"))
                        Data = (parseXMLtoString(evento_estrutura_xml, ".//Data/text()"))

                        if(len(evento_estrutura_dictEventData.keys()) > 1):
                            if(list(evento_estrutura_dictEventData.keys())[1] == "Binary"):
                                Data_Fields = ["Data", "Binary"]
                                Data.append(evento_estrutura_dictEventData.get("Binary"))
                            else:
                                Data_Event_Name = evento_estrutura_dictEventData.get("@Name")

                evento_unico = Event(
                                Filename,
                                ID,
                                SourceName,
                                Provider_Guid,
                                EventID,
                                EventID_Qualifiers,
                                Version,
                                Level,
                                Task,
                                Opcode,
                                Keywords,
                                TimeCreated,
                                EventRecordID,
                                Correlation_ActivityID,
                                Correlation_RelatedActivityID,
                                ProcessID,
                                ThreadID,
                                Channel,
                                Computer,
                                UserID,
                                Data_Event_Name,
                                Data_Fields,
                                Data,
                                UserData_Flag
                            )
                # print(evento_unico)
                listaDe_eventos_dos_arquivos_evtx.append(evento_unico.__dict__)

In [18]:
len(listaDe_eventos_dos_arquivos_evtx)

1122

In [19]:
df_eventos = pd.DataFrame.from_records(listaDe_eventos_dos_arquivos_evtx)
df_eventos.head()

,Filename,ID,SourceName,Provider_Guid,EventID,EventID_Qualifiers,Version,Level,Task,Opcode,...,Correlation_RelatedActivityID,ProcessID,ThreadID,Channel,Computer,UserID,Data_Event_Name,Data_Fields,Data,UserData_Flag
0,Application.evtx,1,VSS,None,13,0,0,2,0,0,...,None,0,0,Application,Athena,None,None,"[Data, Binary]","[{4e14fba2-2e22-11d1-9964-00c04fbbb345},CEvent...",0
1,Application.evtx,2,VSS,None,8193,0,0,2,0,0,...,None,0,0,Application,Athena,None,None,"[Data, Binary]","[CoCreateInstance,0x8007045b, A system shutdow...",0
2,Application.evtx,3,VSS,None,13,0,0,2,0,0,...,None,0,0,Application,Athena,None,None,"[Data, Binary]","[{4e14fba2-2e22-11d1-9964-00c04fbbb345},CEvent...",0
3,Application.evtx,4,VSS,None,8193,0,0,2,0,0,...,None,0,0,Application,Athena,None,None,"[Data, Binary]","[CoCreateInstance,0x8007045b, A system shutdow...",0
4,Application.evtx,5,Microsoft-Windows-Defrag,None,264,32768,0,2,0,0,...,None,0,0,Application,Athena,None,None,"[Data, Binary]","[retrim,Data (D:),The operation requested is n...",0


In [20]:
df_eventos.tail()

,Filename,ID,SourceName,Provider_Guid,EventID,EventID_Qualifiers,Version,Level,Task,Opcode,...,Correlation_RelatedActivityID,ProcessID,ThreadID,Channel,Computer,UserID,Data_Event_Name,Data_Fields,Data,UserData_Flag
1117,System.evtx,1119,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},7000,49152,0,2,0,0,...,None,960,4220,System,Athena,None,None,"[Data, Binary]","[LMS, %%2, 4C004D0053000000]",0
1118,System.evtx,1120,VBoxNetLwf,None,12,49156,0,2,0,0,...,None,4,168,System,Athena,None,None,"[Data, Binary]","[\Device\VBoxNetLwf, 0000000001000000000000000...",0
1119,System.evtx,1121,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},7000,49152,0,2,0,0,...,None,956,4024,System,Athena,None,None,"[Data, Binary]","[LMS, %%2, 4C004D0053000000]",0
1120,System.evtx,1122,VBoxNetLwf,None,12,49156,0,2,0,0,...,None,4,264,System,Athena,None,None,"[Data, Binary]","[\Device\VBoxNetLwf, 0000000001000000000000000...",0
1121,System.evtx,1123,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},7000,49152,0,2,0,0,...,None,952,4040,System,Athena,None,None,"[Data, Binary]","[LMS, %%2, 4C004D0053000000]",0
